In [1]:
from bs4 import BeautifulSoup
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

In [2]:
import re
import ner
import wget
import gensim
import requests
import pymorphy2
from bs4 import BeautifulSoup
from nltk.corpus import stopwords, RegexpTokenizer

In [3]:
def model_init(): # функиция обучения модели
    model_url = 'http://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz'
    modelfile = wget.download(model_url)
    m = 'taiga_upos_skipgram_300_2_2018.vec.gz'
    if m.endswith('.vec.gz'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
    elif m.endswith('.bin.gz'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
    else:
        model = gensim.models.Word2Vec.load(m)
    model.init_sims(replace=True)
    return model

In [4]:
model = model_init() #обучаем модель

In [5]:
#наши ключевые слова 
WORDS = ['сделка', 'стартап', 'инвестор', 'доля']

In [6]:
# запрашиваем у модели синонимы к нашему слову - 10 штук
b = []
for i, v in enumerate(WORDS):
  b.append(model.most_similar(WORDS[i] + '_NOUN'))

/home/timoxinda/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [7]:
# вытаскиваем из модели нужные нам синонимы без лишнего _ NOUN
similar = {}
for i, x in enumerate(b):
  v = []
  for a in x:
    v.append(a[0].split('_')[0])
  similar[WORDS[i]] = v

In [8]:
similar

{'сделка': ['сделк',
  'сделка',
  'сделок',
  'контракт',
  'договор',
  'афера',
  'афёра',
  'соглашение',
  'договора',
  'дельце'],
 'стартап': ['венчурный',
  'бизнес',
  'бизнес-проект',
  'краудфандинг',
  'микрософт',
  'инвестор',
  'корпорация',
  'инновация',
  'инновационный',
  'млм'],
 'инвестор': ['девелопер',
  'инвестиция',
  'застройщик',
  'акционер',
  'рекламодатель',
  'вкладчик',
  'спонсор',
  'предприниматель',
  'кредитор',
  'работодатель'],
 'доля': ['доль',
  'долю',
  'толика',
  'процент',
  'доле',
  'дуль',
  'секундыотреть',
  'доть',
  '9процент-знак',
  'полпроцент']}

In [9]:
k = 19
for i in similar:
    for j in similar[i]:
        if (k == 0):
            break
        page_link = 'https://yandex.ru/' + 'search/?text='+str(j) + '&lr=&clid=-002&win=215'
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        obj = soup.find('span', attrs = {'class':'extended-text__short'})
        print(soup)
        k = k - 1

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Ой!</title><script>;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);</script><!--[if gt IE 8]><!--><link href="captcha.min.css" rel="stylesheet"/><!--<![endif]--><!--[if IE 8]><link rel="stylesheet" href="captcha.ie.min.css"/><![endif]--></head><body class="b-page b-page_type_default b-page__body i-ua i-global i-bem" data-bem='{"i-ua":{},"i-global":{"lang":"ru","tl

In [10]:
page_link = 'https://yandex.ru/' + 'search/?text='+'ВКОНТАК' + '&lr=213&clid=2256434-002&win=215'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [11]:
html = response.content
soup = BeautifulSoup(html,'html.parser')

In [12]:
obj = soup.find('div', attrs = {'class':'home-arrow__sample'})

In [ ]:
soup

In [15]:
import yandex_search

In [16]:
import googlesearch

In [14]:
for x in googlesearch.search(query="сделка", num=10, pause=20, tbs="tbs=cdr:1,cd_min:1/1/2018,cd_max:12/31/2018"):
    print(x)

https://ru.wikipedia.org/wiki/%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0
https://ru.wikipedia.org/wiki/%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0#%D0%A3%D1%81%D0%BB%D0%BE%D0%B2%D0%B8%D1%8F_%D0%B4%D0%B5%D0%B9%D1%81%D1%82%D0%B2%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%81%D0%B4%D0%B5%D0%BB%D0%BA%D0%B8
https://ru.wikipedia.org/wiki/%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0#%D0%92%D0%B8%D0%B4%D1%8B_%D1%81%D0%B4%D0%B5%D0%BB%D0%BE%D0%BA
https://ru.wikipedia.org/wiki/%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0#%D0%A3%D1%81%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5_%D0%B8_%D0%B1%D0%B5%D0%B7%D1%83%D1%81%D0%BB%D0%BE%D0%B2%D0%BD%D1%8B%D0%B5_%D1%81%D0%B4%D0%B5%D0%BB%D0%BA%D0%B8
https://ru.wikipedia.org/wiki/%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0#%D0%A4%D0%BE%D1%80%D0%BC%D0%B0_%D1%81%D0%B4%D0%B5%D0%BB%D0%BA%D0%B8
https://dic.academic.ru/dic.nsf/fin_enc/17826
https://dic.academic.ru/dic.nsf/ushakov/1019878
http://btimes.ru/dictionary/sdelka
https://ru.wiktionary.org/wiki/%D1%81%D0%B4%D0%B5%D0%BB%D0%BA%D0%B0
h

KeyboardInterrupt: 